In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from keras.backend import tensorflow_backend
from keras.utils import to_categorical
from keras.layers.core import K
# K.set_learning_phase(0)

# config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
# session = tf.Session(config=config)
# tensorflow_backend.set_session(session)

Using TensorFlow backend.


In [2]:
def LoadData():
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    Y_train = np.array(train['label'])
    Y_train_cat = to_categorical(Y_train)
    
    X = np.array(train['feature'])
    X_train = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_train.append(x)
    X_train = np.array(X_train)
    X_train = X_train.reshape(X_train.shape[0],48,48,1)/255
    
    X = np.array(test['feature'])
    X_test = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_test.append(x)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0],48,48,1)/255
    
    return X_train, Y_train, X_test, Y_train_cat

In [3]:
X_train, Y_train, X_test, Y_train_cat = LoadData()

In [4]:
# modellist = ['model/model17-0.66.h5', 'model/model23.h5']# 'model/model21-0.6853'
# weight = [0.66 , 0.672]
# modellist += ['./bag/0.h5', './bag/8.h5', 'model/model27.h5', 'model/model28.h5', 'model/model30.h5', 'model/model29.h5' ]
# weight += [ 0.67, 0.67, 0.6853 - 0.01, 0.68681 + 0.01 , 0.66, 0.678] 

# modellist = ['model/model17-0.66.h5', 'model/model23.h5']# 'model/model21-0.6853'
# weight = [0.66 , 0.672]
modellist = []
weight = []

modellist += ['./bag/0.h5', './bag/8.h5', 'model/model27.h5', 'model/model28.h5', 'model/model30.h5']#, 'model/model29.h5' ]
weight += [ 0.67, 0.67, 0.6853, 0.68681 , 0.66]#, 0.678] 

modellist += ['model/model31.h5']
weight += [0.672]

# weight = [0.6820575627679119,
#          0.6725045926572353,
#          0.6750765464018257,
#          0.6754439680472665,
#          0.6756889159739273,
#          0.663686466596642,
#          0.6813065819177938,
#          0.6777709738359959,
#          0.6860992041162677,
#          0.6797353590682744]

# modellist = []
# for i in range(10):
#     modellist.append("./bag/" + str(i) + ".h5")
# print(modellist)

final = np.array([0.0] * 7178 * 7).reshape(7178,7)

# outfile = "vote8"

for i in range(len(modellist)):
    model = load_model(modellist[i])
    predict = model.predict(X_train, batch_size=128)
    final += predict * weight[i]
    del model

final_class = []
for i in range(len(final)):
    final_class.append(np.argmax(final[i]))


# ans = []
# for i in range(len(final_class)):
#     ans.append([i,final_class[i]])

# ans = pd.DataFrame(ans,columns=['id', 'label'])
# ans.to_csv('data/'+ outfile +'.csv', index=False)

ResourceExhaustedError: OOM when allocating tensor with shape[128,48,48,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_58/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_conv2d_58_input_0_1/_1037, conv2d_58/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'conv2d_58/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitle

In [ ]:
final_class